# S11 — Proxecto final (II): versión 1 + iteración e melloras
**Curso:** Programación Musical con Python — 2º Trimestre  
**Duración:** 1 sesión (≈ 1 hora)

## 🎯 Obxectivos
- Converter o MVP de S10 nunha **versión 1** (máis completa e limpa).
- Modularizar o código: funcións pequenas e reutilizables.
- Gardar resultados (MIDI/XML/WAV/PNG) e deixalos organizados.
- Engadir unha capa de **avaliación**: comparación, métricas, ou “tests” manuais.
- Preparar a presentación final de S12.

> 🔁 Hoxe o importante é o **proceso**: probar, mellorar, documentar.


## 0) Checklist da sesión (o que debe quedar feito)

✅ O notebook executa de principio a fin sen erros  
✅ Código organizado por seccións (ler como partitura)  
✅ Funcións pequenas (evitar repetir código)  
✅ Resultados gardados (polo menos 2 outputs)  
✅ Un bloque de “avaliación” (métricas, comparación ou validación)  
✅ Nota final: “que mellorei hoxe” (5 liñas)

> Isto conta como **10% do proxecto final** (versión 1).


## 1) Escolle o teu camiño (A/B/C)

### A) Simbólico (music21)
- Engade variacións novas (transposición, inversión, cambio rítmico)
- Mellora a forma (A–A2–B–A) ou tema+variacións
- Exporta MIDI + MusicXML

### B) Audio (librosa)
- Mellora features + comparación
- Engade espectrograma e BPM/onsets
- Exporta WAV (fragmentos) + gardar gráficas como PNG

### C) Exploración
- Mellora regras / thresholds
- Engade un “dataset” pequeno (5–10 fragmentos)
- Fai unha táboa de resultados e unha conclusión

> Recomendación: mantén o proxecto pequeno pero ben acabado.


## 2) Estrutura recomendada do notebook (copia esta plantilla)

- `# PROPOSTA (resumo)`
- `# IMPORTS E SETUP`
- `# DATOS`
- `# FUNCIÓNS`
- `# PIPELINE PRINCIPAL (versión 1)`
- `# AVALIACIÓN / COMPARACIÓN`
- `# EXPORTACIÓN DE RESULTADOS`
- `# CONCLUSIÓNS + PRÓXIMOS PASOS`


## 3) Plantilla técnica común (utilidades)

Estas funcións axúdanche a gardar outputs e manter orde.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

OUTPUT_DIR = "outputs_s11"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def gardar_fig(nome):
    path = os.path.join(OUTPUT_DIR, nome)
    plt.savefig(path, dpi=150, bbox_inches="tight")
    print("✅ Gardado:", path)
    return path

print("✅ Carpeta outputs:", OUTPUT_DIR)


## 4A) Se o teu proxecto é simbólico (music21) — plantilla versión 1
Inclúe:
- funcións para crear seccións
- polo menos 2 variacións
- exportación


In [ ]:
# --- PROXECTO A (Simbólico) ---
# Se non é o teu proxecto, podes ignorar esta sección.

!pip -q install music21
from music21 import stream, note, meter, tempo, key

def crear_base(time_sig="4/4", bpm=100, tonalidade="C"):
    s = stream.Stream()
    s.append(meter.TimeSignature(time_sig))
    s.append(tempo.MetronomeMark(number=bpm))
    s.append(key.Key(tonalidade))
    return s

def frase(alturas, ritmos):
    s = stream.Stream()
    for a, d in zip(alturas, ritmos):
        if a == "REST":
            s.append(note.Rest(quarterLength=d))
        else:
            s.append(note.Note(a, quarterLength=d))
    return s

def transpoñer(alturas, semitonos):
    out = []
    for a in alturas:
        if a == "REST":
            out.append(a)
        else:
            n = note.Note(a)
            n.transpose(semitonos, inPlace=True)
            out.append(n.nameWithOctave)
    return out

def inverter_contorno(alturas, centro="C4"):
    # Inversión aproximada arredor dun centro
    c = note.Note(centro).pitch.midi
    out = []
    for a in alturas:
        if a == "REST":
            out.append(a)
        else:
            m = note.Note(a).pitch.midi
            inv = c - (m - c)
            out.append(note.Note(inv).nameWithOctave)
    return out

# DATOS
A_alt = ["C4","D4","E4","G4","E4","D4","C4","REST"]
A_rit = [1]*8

# VARIACIÓNS
A_tr = transpoñer(A_alt, 2)
A_inv = inverter_contorno(A_alt, centro="E4")
A_rit_var = [0.5,0.5,1,0.5,0.5,1,2,2]  # suma 8

# PIPELINE (versión 1)
score = crear_base(bpm=110, tonalidade="C")
for sec in [frase(A_alt, A_rit), frase(A_tr, A_rit), frase(A_inv, A_rit_var)]:
    for el in sec.recurse().notesAndRests:
        score.append(el)

score.show("text")

# EXPORTACIÓN
midi_path = score.write("midi", fp=os.path.join(OUTPUT_DIR, "proxectoA_v1.mid"))
xml_path = score.write("musicxml", fp=os.path.join(OUTPUT_DIR, "proxectoA_v1.musicxml"))
midi_path, xml_path


## 4B) Se o teu proxecto é audio (librosa) — plantilla versión 1
Inclúe:
- espectrograma
- BPM + onsets
- features + fingerprint
- gardar 1–2 gráficas como PNG


In [ ]:
# --- PROXECTO B (Audio) ---
# Se non é o teu proxecto, podes ignorar esta sección.

!pip -q install librosa soundfile
import librosa, librosa.display
import soundfile as sf
from IPython.display import Audio, display

def cargar_exemplo(nome):
    p = librosa.ex(nome)
    y, sr = librosa.load(p, sr=None)
    return y, sr

def cortar(y, sr, a, b):
    return y[int(a*sr):int(b*sr)]

def fingerprint(y, sr):
    rms = librosa.feature.rms(y=y)[0]
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    return np.array([np.mean(rms), np.std(rms), np.mean(cent), np.std(cent), np.mean(zcr), np.std(zcr)], float)

def dist(a, b):
    return float(np.linalg.norm(a-b))

# DATOS
y, sr = cargar_exemplo("choice")
frag1 = cortar(y, sr, 0, 6)
frag2 = cortar(y, sr, 6, 12)

display(Audio(frag1, rate=sr))
display(Audio(frag2, rate=sr))

# PIPELINE
bpm1, _ = librosa.beat.beat_track(y=frag1, sr=sr)
bpm2, _ = librosa.beat.beat_track(y=frag2, sr=sr)

on1 = librosa.onset.onset_detect(y=frag1, sr=sr)
on2 = librosa.onset.onset_detect(y=frag2, sr=sr)

fp1, fp2 = fingerprint(frag1, sr), fingerprint(frag2, sr)
print("BPM:", round(float(bpm1),1), round(float(bpm2),1))
print("Onsets:", len(on1), len(on2))
print("Distancia fingerprint:", round(dist(fp1, fp2), 4))

# ESPECTROGRAMA (frag1)
S = np.abs(librosa.stft(frag1))
S_db = librosa.amplitude_to_db(S, ref=np.max)

plt.figure()
librosa.display.specshow(S_db, sr=sr, x_axis="time", y_axis="hz")
plt.colorbar(format="%+2.0f dB")
plt.title("Espectrograma frag1")
guardar_fig("espectrograma_frag1.png")
plt.show()

# EXPORTACIÓN WAV
sf.write(os.path.join(OUTPUT_DIR, "frag1.wav"), frag1, sr)
sf.write(os.path.join(OUTPUT_DIR, "frag2.wav"), frag2, sr)
print("✅ WAV gardados en", OUTPUT_DIR)


## 5) AVALIACIÓN / COMPARACIÓN (obrigatorio)

Escolle unha:
- Comparar dúas versións (antes/despois)
- Comparar dúas melodías/audios con métricas
- Validación manual: “cumpre as regras?”

Escribe 5–8 liñas explicando que significa o resultado.


In [ ]:
# ✅ Espazo para a túa avaliación (exemplo de texto en variables)
avaliacion_texto = """
- Métrica 1: ...
- Métrica 2: ...
- Interpretación: ...
"""
print(avaliacion_texto)


## 6) Conclusións (5 liñas) + próximos pasos (S12)

Escribe:
- Que mellorei hoxe?
- Que falta para rematar?
- Que vou presentar en S12?


### ✍️ Conclusións (copia e completa)

**Que mellorei hoxe:**  
**O maior problema que resolvín:**  
**O que falta para S12:**  
**O que vou ensinar/poñer a soar na presentación final:**


## 🧠 Ler código como partitura (checkpoint)
Antes de entregar:
- Seccións claras con títulos
- Non repetir código: usar funcións
- Deixar outputs gardados en `outputs_s11/`
- Se alguén abre o notebook, debe entender o proxecto en 2 minutos
